In [1]:
import pandas as pd
import numpy as np

In [2]:
# Consider proteins 0 to 4 in rows and TCRs in columns
tpmat = np.array([
    [0.9, 0.7, 0.5, 0.3],
    [1.0, 0.9, 0.8, 0.7],
    [0.9, 0.8, 0.7, 0.99999],
    [1.0, 0.8, 0.6, 0.4],
    [1.0, 0.85, 0.84, 0.83],
    [0.7, 0.2, 0.89999, 0.1],
    [0.9, 0.75, 0.74, 0.73],
    [0.9, 0.85, 0.80, 0.75],
    #[0.7, 0.45, 0.44, 0.43],
    [0.84, 0.85, 0.5, 0.83],
    [0.9, 0.9, 0.6, 0.5],
    [0.6, 0.9, 0.9, 0.4],
])
print(f"{tpmat[1,1]=}")
assert tpmat[1,2] == 0.8
df = pd.DataFrame(tpmat)
df


tpmat[1,1]=0.9


,0,1,2,3
0,0.90,0.70,0.50000,0.30000
1,1.00,0.90,0.80000,0.70000
2,0.90,0.80,0.70000,0.99999
3,1.00,0.80,0.60000,0.40000
4,1.00,0.85,0.84000,0.83000
5,0.70,0.20,0.89999,0.10000
6,0.90,0.75,0.74000,0.73000
7,0.90,0.85,0.80000,0.75000
8,0.84,0.85,0.50000,0.83000
9,0.90,0.90,0.60000,0.50000


In [3]:
def gap_hard(b, p):
    """ return vector gap(t) = 0 if p is not Top1, else Top1-Top2 """
    # this gap is to be maximized over t
    #for t in range(b.shape[0]):
    #    print(f"b[t,:]={b[t,:]} gaps={b[t,:]-b[t,p]} maxgap {np.argmax(b[t,:]-b[t,p])}")

    itop2 = np.argpartition(-tpmat,2, axis=1)[:,:2]
    #print(f"{itop2=}")
    tkax = np.take_along_axis(tpmat,itop2, axis=1)
    #print(f"{tkax=}")

    ptop1 = (tkax[:,0] == b[:,p])
    #print(f"{ptop1=}")
    gaps = tkax[:,0] - tkax[:,1]
    #print(f"{gaps=}")
    ret = np.zeros_like(gaps)
    #print(f"{gaps[ptop1]=}")
    #print(f"{b[ptop1,:]=}")
    ret[ptop1] = gaps[ptop1]
    return ret

print(f"\nGAP HARD")
ret = gap_hard(tpmat,1)
print(f"hard {ret=}")


GAP HARD
hard ret=array([0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ])


In [4]:
def opt_hard_vec(b):
    """ For every t, return (pTop, Top1, Top2) """
    itop2 = np.argpartition(-tpmat,2, axis=1)[:,:2]
    print(f"{itop2=}")
    tkax = np.take_along_axis(tpmat,itop2, axis=1)
    print(f"{tkax=}")
    gaps = tkax[:,0] - tkax[:,1]
    print(f"{gaps=}")
    return itop2[:,0], tkax[:,0], tkax[:,1]

print(f"\nOPT HARD VEC")
p,t1,t2 = opt_hard_vec(tpmat)
print(f"opt_hard_vec {p=} gap={t1-t2}")


OPT HARD VEC
itop2=array([[0, 1],
       [0, 1],
       [3, 0],
       [0, 1],
       [0, 1],
       [2, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 2]])
tkax=array([[0.9    , 0.7    ],
       [1.     , 0.9    ],
       [0.99999, 0.9    ],
       [1.     , 0.8    ],
       [1.     , 0.85   ],
       [0.89999, 0.7    ],
       [0.9    , 0.75   ],
       [0.9    , 0.85   ],
       [0.85   , 0.84   ],
       [0.9    , 0.9    ],
       [0.9    , 0.9    ]])
gaps=array([0.2    , 0.1    , 0.09999, 0.2    , 0.15   , 0.19999, 0.15   ,
       0.05   , 0.01   , 0.     , 0.     ])
opt_hard_vec p=array([0, 0, 3, 0, 0, 2, 0, 0, 1, 0, 1]) gap=[0.2     0.1     0.09999 0.2     0.15    0.19999 0.15    0.05    0.01
 0.      0.     ]


In [5]:
def opt_hard(b, eps=1.e-4):
    """ return (gapmax, tTop, pTop) with gap >= gapmax-eps.
    
    Sort return values by descending binding affinity. """
    itop2 = np.argpartition(-tpmat,2, axis=1)[:,:2]
    #print(f"{itop2=}")
    tkax = np.take_along_axis(tpmat,itop2, axis=1)
    print(f"{tkax=}")
    gaps = tkax[:,0] - tkax[:,1]
    print(f"{gaps=}")
    gapmax = np.max(gaps)
    print(f"{gapmax=}")
    # generate eps-set of high-gap choices for t
    tmax = np.argwhere(gaps >= gapmax - eps)
    #print(f"{tmax=}")
    tmax = tmax[:,0]
    #print(f"{tmax=}")
    pmax = itop2[tmax,0]
    print(f"{tmax=} {pmax=}")
    print(f"best rows: {b[tmax,:]=}")
    if False and tmax.size > 1:
        # Break tied gaps tmax (if possible) for highest Top1 binding
        ptop = tkax[tmax,pmax]
        #print(f"{ptop=}")
        ptopimax = np.argmax(ptop)
        ptopmax = ptop[ptopimax]
        print(f"{ptop=} {ptopimax=} {ptopmax=}")
        # expand ptopmax to an eps-range
        ptopeps = np.argwhere(ptop > ptopmax - eps)[:,0]
        print(f"{ptopeps=}")
        # restrict return values to (t,p of max gap) of high b[t,p]
        tmax = tmax[ptopeps]
        pmax = pmax[ptopeps]

    # Sort tmax,pmax by descending b[tmax,pmax]
    #print(f"{b[tmax,pmax]=}")
    tight = np.argsort(-b[tmax,pmax], kind='stable')
    print(f"{tight=}")
    tmax = tmax[tight]
    pmax = pmax[tight]
    print(f"{tmax=} {pmax=}")
        
    return gapmax, tmax, pmax

print(f"\nOPT HARD")
gap,t,p = opt_hard(tpmat)
print(f"{gap=}\n{t=}\n{p=}")
print(f"descending: {tpmat[t,p]=}") # guaranteed descending binding affinity


OPT HARD
tkax=array([[0.9    , 0.7    ],
       [1.     , 0.9    ],
       [0.99999, 0.9    ],
       [1.     , 0.8    ],
       [1.     , 0.85   ],
       [0.89999, 0.7    ],
       [0.9    , 0.75   ],
       [0.9    , 0.85   ],
       [0.85   , 0.84   ],
       [0.9    , 0.9    ],
       [0.9    , 0.9    ]])
gaps=array([0.2    , 0.1    , 0.09999, 0.2    , 0.15   , 0.19999, 0.15   ,
       0.05   , 0.01   , 0.     , 0.     ])
gapmax=0.20000000000000007
tmax=array([0, 3, 5]) pmax=array([0, 0, 2])
best rows: b[tmax,:]=array([[0.9    , 0.7    , 0.5    , 0.3    ],
       [1.     , 0.8    , 0.6    , 0.4    ],
       [0.7    , 0.2    , 0.89999, 0.1    ]])
tight=array([1, 0, 2])
tmax=array([3, 0, 5]) pmax=array([0, 0, 2])
gap=0.20000000000000007
t=array([3, 0, 5])
p=array([0, 0, 2])
descending: tpmat[t,p]=array([1.     , 0.9    , 0.89999])


In [6]:
def gap_soft(b, p, db, verbose=True):
    """ return gap(t) = 0 if p is not Top1, else Top1 - Avg(b>Top1-db) """
    # this gap is to be maximized over t
    # focus on subset stop of rows sel where p is tightest-binding
    bmax = np.max(b, axis=1)
    print(f"{bmax=}")
    ptop1 = (b[:,p] == bmax)
    print(f"{ptop1=}")
    sel = b[ptop1,:]
    print(f"{sel=}")
    stop = bmax[ptop1]
    print(f"{stop=}")
    # In subset stop, calculate average within-db-of-max binding, selval
    seldb = (sel >= (stop - db)[:,None])
    #print(f"{seldb=}")
    #navg = np.sum(seldb, axis=1)
    #print(f"{navg=}")
    selavg, navg = np.average(sel, axis=1, weights=seldb, returned=True) # (row-avgs, navg)
    #selavg = np.average(sel, axis=1, weights=seldb) # just return row-avg
    #print(f"{selavg=}")
    if verbose: print(f"{selavg=}")
    if verbose: print(f"{navg=}")
    softgaps = stop - selavg
    if verbose: print(f"{softgaps=}")
    # if navg==1, those zero gaps are replaced with a Top1 - Top2 >= db value
    navg1 = (navg == 1.0)
    if verbose: print(f"{navg1=}")
    n0 = seldb[navg1]
    if verbose: print(f"{n0.size=} {n0=}")
    if verbose: print(f"{softgaps[navg1]=}")
    if n0.size:
        itop2 = np.argpartition(-sel,2, axis=1)[:,:2]
        #if verbose: print(f"{itop2=}")
        tkax = np.take_along_axis(sel,itop2, axis=1)
        #if verbose: print(f"{tkax=}")
        gaps = tkax[:,0] - tkax[:,1]
        #if verbose:
        #    tmax = np.argwhere(navg1)[:,0]
        #    print(f"hard {tmax=}")
        #    print(f"{gaps=}")
        softgaps[navg1] = gaps[navg1]
        if verbose: print(f"{softgaps=}")
    
    # now return zeros, except for gaps stop - selavg
    # OHOH : if navg was 1, then soft gap should REVERT to be Top1 - Top2
    #ret = np.zeros_like(bmax)
    ret = np.empty(bmax.shape)
    ret.fill(-1.0)
    #tgaps = b[:,p] - bmax
    #tgaps[ptop1] = stop - selavg
    #print(f"{tgaps=}")
    #ret[ptop1] = stop - selavg
    ret[ptop1] = softgaps
    print(f"{ret=}")
    return ret

print(f"\nGAP SOFT")
p=1; db=0.11; ret = gap_soft(tpmat,p,db)
print(f"soft {p} {db} {ret=}\n")
p=0; db=0.11; ret = gap_soft(tpmat,p,db)
print(f"soft {p} {db} {ret=}\n")
display(df)
p=1; db=0.31; ret = gap_soft(tpmat,p,db)
print(f"soft {p} {db} {ret=}\n")



GAP SOFT
bmax=array([0.9    , 1.     , 0.99999, 1.     , 1.     , 0.89999, 0.9    ,
       0.9    , 0.85   , 0.9    , 0.9    ])
ptop1=array([False, False, False, False, False, False, False, False,  True,
        True,  True])
sel=array([[0.84, 0.85, 0.5 , 0.83],
       [0.9 , 0.9 , 0.6 , 0.5 ],
       [0.6 , 0.9 , 0.9 , 0.4 ]])
stop=array([0.85, 0.9 , 0.9 ])
selavg=array([0.84, 0.9 , 0.9 ])
navg=array([3., 2., 2.])
softgaps=array([0.01, 0.  , 0.  ])
navg1=array([False, False, False])
n0.size=0 n0=array([], shape=(0, 4), dtype=bool)
softgaps[navg1]=array([], dtype=float64)
ret=array([-1.  , -1.  , -1.  , -1.  , -1.  , -1.  , -1.  , -1.  ,  0.01,
        0.  ,  0.  ])
soft 1 0.11 ret=array([-1.  , -1.  , -1.  , -1.  , -1.  , -1.  , -1.  , -1.  ,  0.01,
        0.  ,  0.  ])

bmax=array([0.9    , 1.     , 0.99999, 1.     , 1.     , 0.89999, 0.9    ,
       0.9    , 0.85   , 0.9    , 0.9    ])
ptop1=array([ True,  True, False,  True,  True, False,  True,  True, False,
        True, False]

,0,1,2,3
0,0.90,0.70,0.50000,0.30000
1,1.00,0.90,0.80000,0.70000
2,0.90,0.80,0.70000,0.99999
3,1.00,0.80,0.60000,0.40000
4,1.00,0.85,0.84000,0.83000
5,0.70,0.20,0.89999,0.10000
6,0.90,0.75,0.74000,0.73000
7,0.90,0.85,0.80000,0.75000
8,0.84,0.85,0.50000,0.83000
9,0.90,0.90,0.60000,0.50000


bmax=array([0.9    , 1.     , 0.99999, 1.     , 1.     , 0.89999, 0.9    ,
       0.9    , 0.85   , 0.9    , 0.9    ])
ptop1=array([False, False, False, False, False, False, False, False,  True,
        True,  True])
sel=array([[0.84, 0.85, 0.5 , 0.83],
       [0.9 , 0.9 , 0.6 , 0.5 ],
       [0.6 , 0.9 , 0.9 , 0.4 ]])
stop=array([0.85, 0.9 , 0.9 ])
selavg=array([0.84, 0.8 , 0.8 ])
navg=array([3., 3., 3.])
softgaps=array([0.01, 0.1 , 0.1 ])
navg1=array([False, False, False])
n0.size=0 n0=array([], shape=(0, 4), dtype=bool)
softgaps[navg1]=array([], dtype=float64)
ret=array([-1.  , -1.  , -1.  , -1.  , -1.  , -1.  , -1.  , -1.  ,  0.01,
        0.1 ,  0.1 ])
soft 1 0.31 ret=array([-1.  , -1.  , -1.  , -1.  , -1.  , -1.  , -1.  , -1.  ,  0.01,
        0.1 ,  0.1 ])



In [7]:
def gap_soft_vec(b, db=0.1, eps=1.e-4, verbose=False):
    """ For every t, return [pTop1], Avg(b>Top1-db) """
    pmax = np.argmax(b, axis=1)
    bmax = np.take_along_axis(b, pmax[:,None], axis=1)[:,0]
    if verbose: print(f"{pmax=}\n{bmax=}")
    dbthresh = bmax - db
    #if verbose: print(f"{dbthresh=}")
    window = (b >= dbthresh[:,None])
    #if verbose: print(f"{window=}")
    selavg, navg = np.average(b, axis=1, weights=window, returned=True) # (row-avgs, navg)
    #selavg = np.average(b, axis=1, weights=window) # just return row-avg
    #if verbose: print(f"{selavg=}")
    softgaps = bmax - selavg
    if verbose: print(f"{softgaps=}")
    if verbose: print(f"{navg=}")
    # if navg==1, those zero gaps are replaced with a Top1 - Top2 >= db value
    navg1 = (navg == 1.0)
    n0 = b[navg1]
    if verbose: print(f"{n0.size=} {n0=}")
    if n0.size:
        itop2 = np.argpartition(-n0,2, axis=1)[:,:2]
        #if verbose: print(f"{itop2=}")
        tkax = np.take_along_axis(n0,itop2, axis=1)
        #if verbose: print(f"{tkax=}")
        gaps = tkax[:,0] - tkax[:,1]
        if verbose:
            tmax = np.argwhere(navg1)[:,0]
            print(f"tmax hard {tmax=}")
            print(f"{gaps=}")
        softgaps[navg1] = gaps
        if verbose: print(f"{softgaps=}")
    
    # update pmax to a set of high-score proteins using eps
    # (these all have equivalent gap value)
    bmaxeps = bmax - eps
    if verbose: print(f"{bmaxeps = }")
    #pmaxeps = np.argwhere(b >= bmaxeps[:,None]) # list of index-tuples
    #if verbose: print(f"{pmaxeps=}")
    plist = tuple(np.nonzero(row) for row in (b >= bmaxeps[:,None]))
    # NEW return plist is two same-size int vectors, (indices("t"), indices("p"))
    #plist = np.nonzero(b >= bmaxeps[:,None])
    #if verbose: print(f"{plist=}")
    return plist, softgaps

print(f"\nGAP SOFT VEC")
print(f"{tpmat=}")
plist, gaps = gap_soft_vec(tpmat, 0.11, verbose=True)
print(f"gap_soft_vec *** {plist=}\n{gaps=}\n")

tpmat2 = np.array([
    [0.9, 0.8, 0.5, 0.3],
    [1.0, 0.9, 0.8, 0.7],
    [1.0, 0.9, 0.6, 0.4],
    [0.8, 0.1, 0.1, 0.9],
    [1.0, 0.2, 0.9, 0.1],
    [0.9, 0.85, 0.84, 0.83],
    [0.84, 0.85, 0.83, 0.9],
    [0.84, 0.85, 0.5, 0.83],
    [0.9, 0.9, 0.8, 0.6],
    [0.8, 0.9, 0.9, 0.7],
])
print(f"{tpmat2=}")
plist,gaps = gap_soft_vec(tpmat2, 0.11, verbose=True)
print(f"\n *** {plist=}\n{gaps=}\n")

tpmat3 = np.array([
    [0.90001, 0.9, 0.8, 0.6],
    [0.8, 0.8, 0.7, 0.6],
    [0.8, 0.9, 0.90001, 0.7],
    [0.86, 0.900001, 0.86, 0.1],
    [0.76, 0.1, 0.76, 0.8],
])
print(f"{tpmat3=}")
plist,gaps = gap_soft_vec(tpmat3, 0.11, verbose=True)
print(f"\n *** {plist=}\n{gaps=}\n")



GAP SOFT VEC
tpmat=array([[0.9    , 0.7    , 0.5    , 0.3    ],
       [1.     , 0.9    , 0.8    , 0.7    ],
       [0.9    , 0.8    , 0.7    , 0.99999],
       [1.     , 0.8    , 0.6    , 0.4    ],
       [1.     , 0.85   , 0.84   , 0.83   ],
       [0.7    , 0.2    , 0.89999, 0.1    ],
       [0.9    , 0.75   , 0.74   , 0.73   ],
       [0.9    , 0.85   , 0.8    , 0.75   ],
       [0.84   , 0.85   , 0.5    , 0.83   ],
       [0.9    , 0.9    , 0.6    , 0.5    ],
       [0.6    , 0.9    , 0.9    , 0.4    ]])
pmax=array([0, 0, 3, 0, 0, 2, 0, 0, 1, 0, 1])
bmax=array([0.9    , 1.     , 0.99999, 1.     , 1.     , 0.89999, 0.9    ,
       0.9    , 0.85   , 0.9    , 0.9    ])
softgaps=array([0.      , 0.05    , 0.049995, 0.      , 0.      , 0.      ,
       0.      , 0.05    , 0.01    , 0.      , 0.      ])
navg=array([1., 2., 2., 1., 1., 1., 1., 3., 3., 2., 2.])
n0.size=20 n0=array([[0.9    , 0.7    , 0.5    , 0.3    ],
       [1.     , 0.8    , 0.6    , 0.4    ],
       [1.     , 0.85   

In [8]:
def opt_soft(b, db=0.1, eps=1.e-4, verbose=False):
    """ return highest soft-gaps (with eps slop) and set of highest p (with eps slop). """
    plist, gaps = gap_soft_vec(b, db=db, eps=eps)
    if verbose: print(f"{plist=}")
    if verbose: print(f"{gaps=}")
    gargmax = np.argmax(gaps)
    gmax = gaps[gargmax]
    tmax = np.argwhere(gaps > gmax - eps)[:,0]
    if verbose: print(f"{gmax=} {tmax=}")
    # tmax have eps-equal soft-gap.  sort them by descending max binding affinity b[tmax,pmax]
    brows = b[tmax,:]
    #if verbose: print(f"best rows {brows=}")
    bmax  = np.max(brows, axis=1)
    #if verbose: print(f"{bmax=}")
    torder = np.argsort(-bmax, kind='stable')
    #if verbose: print(f"{torder=}")
    bmax = bmax[torder]
    tmax = tmax[torder]
    if verbose: print(f"{gmax=} {tmax=} {bmax=}")
    if False:
        return gmax, tmax, bmax
    #if verbose: print(f"best rows: {b[tmax,:]=}")
    
    # reorder the interesting subset of plist
    pmaxes = [plist[t] for t in tmax]
    #if verbose: print(f"{pmaxes=}")

    if True: # for returning plain list of (t,p) index-tuples as matrix(N,2)
        #
        # This is easier than always calling tmax_plist_to_tp_array(tmax, pmaxes) using our return values
        #
        repeats = [len(pp[0]) for pp in pmaxes]
        #repeats = [len(plist[t][0]) for t in tmax] # equivalent
        #print(f"{repeats=}")
        #print(f"{tmax=}")
        tmaxrep = np.repeat(tmax, repeats)
        if verbose: print(f"{tmaxrep=}")

        #bmaxeps = bmax - eps
        #if verbose: print(f"{bmaxeps = }")
        ##pmaxeps = np.argwhere(b >= bmaxeps[:,None]) # list of index-tuples
        ##if verbose: print(f"{pmaxeps=}")
        #tplist = np.nonzero(b[tmax,:] >= bmaxeps[:,None])
        #if verbose: print(f"{tplist=}")
        #tp = np.hstack((tmaxrep[:,None], tplist[1][:,None]))
        #if verbose: print(f"{tp=}")
        # OR ... simpler ... re-use existing pmaxes ...
        hpmaxes = np.hstack(pmaxes)[0]
        print(f"{hpmaxes=}")
        tp = np.hstack((tmaxrep[:,None], hpmaxes[:,None]))
        if verbose: print(f"{tp=}")

    
    #phigh = b > (b.ma
    if True:
        return gmax, tmax, pmaxes
    else:
        return gmax, tp

def tmax_plist_to_tp_array(tmax,plist):
    tp = np.empty((0,2))
    ###print(f"{tp=}")
    bs = []
    for t,p in zip(tmax,plist):
        a=np.repeat([t],len(p))
        #b=np.hstack((a[:,None],p))
        b = np.meshgrid([t],list(p))
        b = np.hstack(b)
        ###print(f"{t=} {a=} {p=} {b=}")
        #print(f"{t=} {p=} {b=}")
        bs.append(b)

    tp = np.vstack(tuple(bs))
    return tp
    
print(f"\n{tpmat=}")
print(f"{tpmat.max(axis=1)=}")
tpmaxes = tpmat.max(axis=1)
print(f"{tpmat > (tpmaxes - 1e-4)[:,None]=}")

print(f"\nOPT SOFT")
gapmax,tmax,plist = opt_soft(tpmat, 0.11, verbose=True)
print(f"\n *** {gapmax=} {tmax=} {plist=}")
#tp = tmax_plist_to_tp_array(tmax, plist)
#print(f"{tp=}\n")

print(f"\n{tpmat2=}")
print(f"OPT SOFT")
gapmax,tmax,plist = opt_soft(tpmat2, 0.11, verbose=True)
print(f"\n *** {gapmax=} {tmax=} {plist=}")
#tp = tmax_plist_to_tp_array(tmax, plist)
#print(f"{tp=}\n")
    
print(f"\n{tpmat3=}")
print(f"OPT SOFT")
gapmax,tmax,plist = opt_soft(tpmat3, 0.11, verbose=True)
print(f"\n *** {gapmax=} {tmax=} {plist=}")
#tp = tmax_plist_to_tp_array(tmax, plist)
#print(f"{tp=}\n")
#


tpmat=array([[0.9    , 0.7    , 0.5    , 0.3    ],
       [1.     , 0.9    , 0.8    , 0.7    ],
       [0.9    , 0.8    , 0.7    , 0.99999],
       [1.     , 0.8    , 0.6    , 0.4    ],
       [1.     , 0.85   , 0.84   , 0.83   ],
       [0.7    , 0.2    , 0.89999, 0.1    ],
       [0.9    , 0.75   , 0.74   , 0.73   ],
       [0.9    , 0.85   , 0.8    , 0.75   ],
       [0.84   , 0.85   , 0.5    , 0.83   ],
       [0.9    , 0.9    , 0.6    , 0.5    ],
       [0.6    , 0.9    , 0.9    , 0.4    ]])
tpmat.max(axis=1)=array([0.9    , 1.     , 0.99999, 1.     , 1.     , 0.89999, 0.9    ,
       0.9    , 0.85   , 0.9    , 0.9    ])
tpmat > (tpmaxes - 1e-4)[:,None]=array([[ True, False, False, False],
       [ True, False, False, False],
       [False, False, False,  True],
       [ True, False, False, False],
       [ True, False, False, False],
       [False, False,  True, False],
       [ True, False, False, False],
       [ True, False, False, False],
       [False,  True, False, False],

In [9]:
a = np.array([1,2,3])
b = np.repeat(a, [1,2,3])
print(f"{b=}")

b=array([1, 2, 2, 3, 3, 3])


In [10]:
def gap_mm(b,p):
    """ return max_p' (b(t,p') - b(t,p)) (inner part of min-max expression) """
    bnotp = np.delete(b, p, axis=1)
    #print(f"{bnotp=}")
    bp = b[:,p:p+1]
    print(f"{bp[:,0]=}")
    gaps = bnotp - b[:,p:p+1]
    #print(f"{gaps=}")
    ret = np.max(gaps, axis=1)
    #print(f"max gaps {ret=}")
    return ret

print(f"{tpmat=}")
p=0; ret = gap_mm(tpmat,p)
print(f"mm {p} {ret=}")
p=1; ret = gap_mm(tpmat,p)
print(f"mm {p} {ret=}")
p=2; ret = gap_mm(tpmat,p)
print(f"mm {p} {ret=}")
p=3; ret = gap_mm(tpmat,p)
print(f"mm {p} {ret=}")

tpmat=array([[0.9    , 0.7    , 0.5    , 0.3    ],
       [1.     , 0.9    , 0.8    , 0.7    ],
       [0.9    , 0.8    , 0.7    , 0.99999],
       [1.     , 0.8    , 0.6    , 0.4    ],
       [1.     , 0.85   , 0.84   , 0.83   ],
       [0.7    , 0.2    , 0.89999, 0.1    ],
       [0.9    , 0.75   , 0.74   , 0.73   ],
       [0.9    , 0.85   , 0.8    , 0.75   ],
       [0.84   , 0.85   , 0.5    , 0.83   ],
       [0.9    , 0.9    , 0.6    , 0.5    ],
       [0.6    , 0.9    , 0.9    , 0.4    ]])
bp[:,0]=array([0.9 , 1.  , 0.9 , 1.  , 1.  , 0.7 , 0.9 , 0.9 , 0.84, 0.9 , 0.6 ])
mm 0 ret=array([-0.2    , -0.1    ,  0.09999, -0.2    , -0.15   ,  0.19999,
       -0.15   , -0.05   ,  0.01   ,  0.     ,  0.3    ])
bp[:,0]=array([0.7 , 0.9 , 0.8 , 0.8 , 0.85, 0.2 , 0.75, 0.85, 0.85, 0.9 , 0.9 ])
mm 1 ret=array([ 0.2    ,  0.1    ,  0.19999,  0.2    ,  0.15   ,  0.69999,
        0.15   ,  0.05   , -0.01   ,  0.     ,  0.     ])
bp[:,0]=array([0.5    , 0.8    , 0.7    , 0.6    , 0.84   , 0.8999

In [11]:
def gap_mm2(b,p):
    """ return max_p' (b(t,p') - b(t,p)) (inner part of min-max expression) """
    bp = b[:,p:(p+1)]
    print(f"{bp[:,0]=}")
    if True:
        bnotp = np.delete(b, p, axis=1)
        #print(f"{bnotp=}")
        gaps = bnotp - b[:,p:p+1]
        #print(f"{gaps=}")
        ret = np.max(gaps, axis=1)
        print(f"ret {ret}")
        #return ret
    
    print(f"max gaps {ret=}")
    itop2 = np.argpartition(-tpmat,2, axis=1)[:,:2]
    #print(f"{itop2=}")
    print(f"{itop2[:,0]=}")
    #tkax = np.take_along_axis(tpmat,itop2, axis=1)
    #print(f"{tkax=}")
    #itopisp = itop2[:,0]==p
    #print(f"{itopisp=}")
    #choice0 = itop2[:,0]
    #print(f"{choice0=}")
    #choice1 = itop2[:,1]
    #print(f"{choice1=}")
    #imaxnotp = np.select(itopisp, [choice1,choice0])
    #imaxnotp = np.where(itopisp, choice1, choice0)
    choice0 = itop2[:,0]
    imaxnotp = np.where( choice0==p, itop2[:,1], choice0)
    print(f"{imaxnotp=}")
    bpp = np.take_along_axis(b, imaxnotp[:,None], axis=1)[:,0]
    print(f"bp {bp[:,0]}")
    print(f"bpp {bpp}")
    #print(f"diff {bpp-bp[:,0]}")
    ret = bpp-bp[:,0]
    print(f"ret {ret}")
    #gaps = tkax[:,0] - tkax[:,1]
    #print(f"{gaps=}")
    
    return ret

print(f"{tpmat=}")
p=0; ret = gap_mm2(tpmat,p)
print(f"mm {p} {ret=}")
p=1; ret = gap_mm2(tpmat,p)
print(f"mm {p} {ret=}")
p=2; ret = gap_mm2(tpmat,p)
print(f"mm {p} {ret=}")
p=3; ret = gap_mm2(tpmat,p)
print(f"mm {p} {ret=}")

tpmat=array([[0.9    , 0.7    , 0.5    , 0.3    ],
       [1.     , 0.9    , 0.8    , 0.7    ],
       [0.9    , 0.8    , 0.7    , 0.99999],
       [1.     , 0.8    , 0.6    , 0.4    ],
       [1.     , 0.85   , 0.84   , 0.83   ],
       [0.7    , 0.2    , 0.89999, 0.1    ],
       [0.9    , 0.75   , 0.74   , 0.73   ],
       [0.9    , 0.85   , 0.8    , 0.75   ],
       [0.84   , 0.85   , 0.5    , 0.83   ],
       [0.9    , 0.9    , 0.6    , 0.5    ],
       [0.6    , 0.9    , 0.9    , 0.4    ]])
bp[:,0]=array([0.9 , 1.  , 0.9 , 1.  , 1.  , 0.7 , 0.9 , 0.9 , 0.84, 0.9 , 0.6 ])
ret [-0.2     -0.1      0.09999 -0.2     -0.15     0.19999 -0.15    -0.05
  0.01     0.       0.3    ]
max gaps ret=array([-0.2    , -0.1    ,  0.09999, -0.2    , -0.15   ,  0.19999,
       -0.15   , -0.05   ,  0.01   ,  0.     ,  0.3    ])
itop2[:,0]=array([0, 0, 3, 0, 0, 2, 0, 0, 1, 0, 1])
imaxnotp=array([1, 1, 3, 1, 1, 2, 1, 1, 1, 1, 1])
bp [0.9  1.   0.9  1.   1.   0.7  0.9  0.9  0.84 0.9  0.6 ]
bpp [0.7     

In [12]:
#
# Consecutive min_t max_p'!=p b(p',t) - b(p,t) yields the t with the
# lowest range of binding affinity values.
#
# This is NOT the most specific, and perhaps one of the least specific.
#
def gap_mm3(b,eps=1e-4,verbose=True):
    """ return min_t max_p' (b(t,p') - b(t,p)) (inner part of min-max expression) """
    # If b(t,p) is not Top1(t), then max_p' is b(t,Top1(t)) - b(t,p)
    #                                     else b(t,Top2(t)) - b(t,p)
    # so ignoring Top2 branch, min_t is then b(t,Top1(t)) - b(t,p)
    # Now if p is NOT in Top1/Top2, b(t,p) is very low, so irrelevant for the min
    # ...
    itop2 = np.argpartition(-tpmat,2, axis=1)[:,:2]
    #print(f"{itop2=}")
    print(f"{itop2[:,0]=}")
    choice0 = itop2[:,0]
    choice1 = itop2[:,1]
    
    diffs=[]
    for p in range(b.shape[1]):
        bp = b[:,p]
        print(f"======= {p=} {bp=}")
        imaxnotp = np.where( choice0==p, choice1, choice0)
        #print(f"{imaxnotp=}")
        bpp = np.take_along_axis(b, imaxnotp[:,None], axis=1)[:,0]
        #print(f"bpp {bpp}")
        diff = bpp-bp
        #print(f"diff {diff}")
        idmax = np.argmax(diff)
        dmax = diff[idmax]
        print(f"{idmax=} {dmax=}")
        diffs.append(diff)
    
    diffs = np.vstack(diffs) # NB: diffs[p,t] = b[p',t] - b[p,t]
    print(f"{diffs=}")
    # For each p diffs[p,:], where/what is pp = max over p'?
    ippmax = np.argmax(diffs, axis=0)
    ppmax = np.take_along_axis(diffs, ippmax[None,:], axis=0)[0]
    print(f"{ippmax=} {ppmax=}")
    # min over t of ppmax
    itmin = np.argmin(ppmax)
    tmin = ppmax[itmin]
    pp = ippmax[itmin]
    print(f"{itmin=} {tmin=} {pp=}")
    min_maxgap = tmin
    return min_maxgap, pp, itmin
    ## OOPS
    ## max over p' (a function of t), for all p
    #ippmax = np.argmax(diffs, axis=1)
    #ppmax = np.take_along_axis(diffs, ippmax[:,None], axis=1)[:,0]
    #print(f"{ippmax=} {ppmax=}")
    ## final min over t (p)
    #ipp  = np.argmin(ppmax, axis=0)
    #pp = bpp[ipp]
    #print(f"{ipp=} {pp=}")
    #p = ppmax[pp]
    #print(f"{tpp=}")
    #print(f"tp = ({ipp}, {p})")
    #
    #return pp, tipp, p

#print(f"{tpmat.shape=} tpmat=\n{tpmat}")
p=0; ret = gap_mm3(tpmat)
print(f"mm {p} {ret=}")
mmgap,pp,t = ret
print(f"{t=} {p=} {pp=} {mmgap=}")
if False:
    def highlight_t(s, row):
        print(f"{s=} {row=}")
        is_max = pd.Series(data=False, index=s.index)
        is_max['hi'] = (s.loc == 7)
        return ['background-color: yellow' if is_max.any() else '' for v in is_max]
        #if s.index == 7:
        #    return ['background-color: yellow'] * len(s)
        #else:
        #    return ['background-color: white'] * len(s)
    
    # did not seem to work in jupyter notebook:
    df.style.apply(highlight_t, axis=1)

display(df)
print(f"tpmat[t,{pp=}]={tpmat[t,pp]} tpmat[t,{p=}]={tpmat[t,p]}")
#=1; ret = gap_mm3(tpmat)
#rint(f"mm {p} {ret=}")
#=2; ret = gap_mm3(tpmat)
#rint(f"mm {p} {ret=}")
#=3; ret = gap_mm3(tpmat)
#rint(f"mm {p} {ret=}")
#df


itop2[:,0]=array([0, 0, 3, 0, 0, 2, 0, 0, 1, 0, 1])
======= p=0 bp=array([0.9 , 1.  , 0.9 , 1.  , 1.  , 0.7 , 0.9 , 0.9 , 0.84, 0.9 , 0.6 ])
idmax=10 dmax=0.30000000000000004
======= p=1 bp=array([0.7 , 0.9 , 0.8 , 0.8 , 0.85, 0.2 , 0.75, 0.85, 0.85, 0.9 , 0.9 ])
idmax=5 dmax=0.6999899999999999
======= p=2 bp=array([0.5    , 0.8    , 0.7    , 0.6    , 0.84   , 0.89999, 0.74   ,
       0.8    , 0.5    , 0.6    , 0.9    ])
idmax=0 dmax=0.4
======= p=3 bp=array([0.3    , 0.7    , 0.99999, 0.4    , 0.83   , 0.1    , 0.73   ,
       0.75   , 0.83   , 0.5    , 0.4    ])
idmax=5 dmax=0.79999
diffs=array([[-0.2    , -0.1    ,  0.09999, -0.2    , -0.15   ,  0.19999,
        -0.15   , -0.05   ,  0.01   ,  0.     ,  0.3    ],
       [ 0.2    ,  0.1    ,  0.19999,  0.2    ,  0.15   ,  0.69999,
         0.15   ,  0.05   , -0.01   ,  0.     ,  0.     ],
       [ 0.4    ,  0.2    ,  0.29999,  0.4    ,  0.16   , -0.19999,
         0.16   ,  0.1    ,  0.35   ,  0.3    ,  0.     ],
       [ 0.6    ,  0.

,0,1,2,3
0,0.90,0.70,0.50000,0.30000
1,1.00,0.90,0.80000,0.70000
2,0.90,0.80,0.70000,0.99999
3,1.00,0.80,0.60000,0.40000
4,1.00,0.85,0.84000,0.83000
5,0.70,0.20,0.89999,0.10000
6,0.90,0.75,0.74000,0.73000
7,0.90,0.85,0.80000,0.75000
8,0.84,0.85,0.50000,0.83000
9,0.90,0.90,0.60000,0.50000


tpmat[t,pp=3]=0.75 tpmat[t,p=0]=0.9
